# Plateforme Agnostique de Traitement et d'Analyse des Textes
### Carnet d'expérimentation
---

## Sujet : Explainability


---

# Initialisations
---

In [1]:
_rs = 42

In [2]:
cd ../..

/Volumes/Geek/Work/Patat


In [3]:
import ast
import importlib
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
tqdm.pandas()

# Préparation Données

---

## Chargement

In [4]:
import patat.util.file

filename = 'data/prod/230517-OIDS-Label.pickle'

df_label = patat.util.file.pickle_load(filename)

In [5]:
df_label.columns

Index(['url', 'site', 'title', 'article', 'tags', 'paragraphs', 'date_iso',
       'abstract', 'Auteur', 'author', 'date', 'infox', 'justification infox',
       'entites_nommees', 'ouverture_esprit', 'faits', 'opinions',
       'propos_raportes', 'sources_citees', 'fausse_nouvelle', 'insinuations',
       'exageration', 'inverifiable', 'titre_decale', 'cherry_picking',
       'remarque observatoire', 'signe', 'entites_coherentes',
       'qualite_ecriture', 'intention', 'scientifique_sulfureux', 'trop_mots',
       'trop_chiffres', 'sophisme', 'inversion_preuve', 'denigrement',
       'degre_infox', 'observateur', 'source', 'p_count', 'text', 'c_count',
       'p_size', 'url_h'],
      dtype='object')

In [6]:
labels = ['infox',
       'justification infox', 'entites_nommees', 'ouverture_esprit', 'faits',
       'opinions', 'propos_raportes', 'sources_citees', 'fausse_nouvelle',
       'insinuations', 'exageration', 'inverifiable', 'titre_decale',
       'signe',
       'entites_coherentes', 'qualite_ecriture', 'intention',
       'scientifique_sulfureux', 'trop_mots', 'trop_chiffres', 'sophisme',
       'inversion_preuve']

In [7]:
selected_labels = ['infox','opinions','exageration','insinuations']

In [8]:
stopwords = patat.util.file.pickle_load('data/tmp/stopwords.pickle')

## Labels ou textes Nuls

In [ ]:
df_label['text'].isna().sum()

In [ ]:
for label in selected_labels:
    print(f'{label} : {df_label[label].isna().sum()}')

## Données dupliquées

In [ ]:
df_label.duplicated(subset='text').sum()

In [ ]:
df_label.duplicated(subset='url').sum()

# Experience
---

## Equilibrage Dataset

In [ ]:
label_count = df_label['infox'].value_counts()
sns.barplot(x=label_count.index, y=label_count)

In [ ]:
df_0 = df_label[df_label['infox'] == 0]
df_1 = df_label[df_label['infox'] == 1]

min_sample = min(len(df_0),len(df_1))


df_0=df_0.sample(min_sample,random_state=_rs)
df_1=df_1.sample(min_sample,random_state=_rs)
df_ml = pd.concat([df_0,df_1])[['text','infox']]

df_ml

In [ ]:
print(df_ml['text'][15])

In [ ]:
df_ml = df_label

# Experience
---

## Expérience de cross validation pour mesurer la performance des modeles


`scores = cross_validate(classifier, X, y, cv=5, scoring = metriques)`

- `classifier` : Classifier
- `X` : Features
- `y` : Label cible
- `cv` : Nombre de cycle d'entrainement/test
- `scoring` : Métriques





In [ ]:
cv=5
y = df_ml['infox']

## Tfidf 
Features : `X_tfidf`

In [ ]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def tokenizer(t):
    return re.findall(r'[^\W\d][^\W\d]+',t)

In [ ]:
#vectorizer = TfidfVectorizer(stop_words=stopwords, token_pattern=r'[^\W\d][^\W\d]+')
vectorizer = TfidfVectorizer(stop_words=stopwords)

In [ ]:
X_tfidf = vectorizer.fit_transform(df_ml['text'])

## SKLearn Logistic Regression

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
logreg = LogisticRegression(C=150, random_state=_rs, solver='lbfgs', multi_class='ovr', max_iter=1000)

In [ ]:
#svd = TruncatedSVD(n_components=500, n_iter=7, random_state=42)
#X = svd.fit_transform(X_tfidf)
X = X_tfidf

In [ ]:
pd.set_option('display.precision',4)

In [ ]:
df_results = []
for label in selected_labels:
    print(f'Processing : {label}')
    y = df_ml[label]
    y_label = y[y.notna()]
    X_label = X[y_label.index]
    scores = cross_validate(logreg, X_label, y_label, cv=cv,
                        scoring=('accuracy','f1','recall','roc_auc'))    
    df_scores=pd.DataFrame(scores)
    df_means = pd.DataFrame(df_scores.describe().loc['mean'],columns=[label])
    #df_results = df_results.append(df_means)
    print(df_scores)
pd.concat(df_results)

In [ ]:
df_scores.describe().loc['mean']

In [ ]:
pd.DataFrame(df_scores.describe().loc['mean']).rename_axis(columns={'mean':})

In [ ]:
df_scores=pd.DataFrame(scores)
#df_means = pd.DataFrame(df_scores.describe().loc[['mean','std']])
df_means = pd.DataFrame(df_scores.describe().loc['mean'])

pd.set_option('display.precision',4)
df_results = df_means
df_results

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
plt.figure(figsize=(8,6))
ax = sns.barplot(df_results.T)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
plt.figure(figsize=(8,6))
ax = sns.catplot(df_scores[['test_accuracy','test_f1','test_recall','test_roc_auc']])

### Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
predictor_pipe = Pipeline(steps = [
    ('vectorizer',vectorizer), 
    ('classifier',logreg)
                  ])
predictor_pipe.fit(df_ml['text'], y)

# Bricolage

In [9]:
df_label

,url,site,title,article,tags,paragraphs,date_iso,abstract,Auteur,author,...,inversion_preuve,denigrement,degre_infox,observateur,source,p_count,text,c_count,p_size,url_h
0,https://www.alnas.fr/actualite/communaute/lara...,www.alnas.fr,L’Arabie saoudite organise des célébrations d'...,L’Arabie saoudite a organisé un événement pour...,"[Accueil, Actualités, Communauté]",[L’Arabie saoudite a organisé un événement pou...,2022-10-31T00:00:00,,,Par Youcef -,...,0.0,0.0,,FB,Recueil,8,L’Arabie saoudite organise des célébrations d'...,2359,294.875000,ef05018ec0f5d637bceb4cef7004e8ee5fe48f6f
1,https://lesmoutonsrebelles.com/les-reseaux-soc...,lesmoutonsrebelles.com,Les réseaux sociaux coupés volontairement ? Qu...,Il semblerait que la panne mondiale des réseau...,[],[Il semblerait que la panne mondiale des résea...,,,,,...,0.0,NaN,1,FB,Recueil,13,Les réseaux sociaux coupés volontairement ? Qu...,3723,286.384615,78521e5bf45d5d672a403958243a58fd95855660
2,https://lesmoutonsrebelles.com/un-rapport-du-p...,lesmoutonsrebelles.com,Un rapport du Pentagone affirme que les extrat...,Selon une énorme base de données de documents ...,[],[Selon une énorme base de données de documents...,,,,,...,0.0,NaN,1,FB,Recueil,17,Un rapport du Pentagone affirme que les extrat...,3545,208.529412,db5f4fe97288e2cb2982f2f3f084f299b4bac7a0
3,https://lesmoutonsrebelles.com/un-sondage-defa...,lesmoutonsrebelles.com,Un sondage défavorable pour Emmanuel Macron su...,"Des internautes, dont des politiques, ont affi...",[],"[Des internautes, dont des politiques, ont aff...",,,,,...,0.0,NaN,2,FB,Recueil,14,Un sondage défavorable pour Emmanuel Macron su...,1928,137.714286,8189af39fedefc6c901fc8b3a1717fdf5288d7fc
4,https://lemediaen442.fr/le-president-ukrainien...,lemediaen442.fr,Le président ukrainien Zelensky se cache dans ...,Le courage de Zelensky aura été de courte duré...,"[Politique, ukraine, volodymyr zelensky]",[Le courage de Zelensky aura été de courte dur...,2022-03-05T14:55:48+01:00,,,,...,0.0,NaN,3,FB,Recueil,8,Le président ukrainien Zelensky se cache dans ...,2045,255.625000,0c2e41f58ac869eb95865751caa3ab29a9f0cd13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,https://lezarceleurs.blogspot.com/2022/04/des-...,lezarceleurs.blogspot.com,Des victimes de test PCR témoignent,De : https://www.naturalnews.com/2022-04-21-vi...,"[accidents, COVID-19, Test PCR, victimes]",[ De : https://www.naturalnews.com/2022-04-21-...,,,NaN,,...,NaN,NaN,NaN,NaN,Verif,29,Des victimes de test PCR témoignent\n De : ht...,6042,208.344828,e6cf82ff921171ab67ac2b45ce27dca77a397c1a
902,https://www.francesoir.fr/politique-monde/resu...,www.francesoir.fr,Résultats de l’audit en Arizona : 54 000 bulle...,"CHRONIQUE - Le 24 septembre, le rapport d’audi...","[Accueil, Politique]","[CHRONIQUE - Le 24 septembre, le rapport d’aud...",2021-09-29T13:37:00,,NaN,"Auteur(s)\nElisabeth Dancet, pour FranceSoir",...,NaN,NaN,NaN,NaN,Verif,12,Résultats de l’audit en Arizona : 54 000 bulle...,10542,878.500000,c587dbcf33207a0a5c066bcf21cff88cd9fe1c9d
903,https://www.francetvinfo.fr/meteo/orages/tempe...,www.francetvinfo.fr,Tempête Aurore : de puissantes rafales de vent...,Le département du Finistère a été frappé par l...,[],[Le département du Finistère a été frappé par ...,2021-10-21T22:53:00,La moitié nord de la France a été frappée par ...,NaN,"H. Puffeney, A.Lay, J. Jonas, J. Chouquet, Fra...",...,NaN,NaN,NaN,NaN,Verif,2,Tempête Aurore : de puissantes rafales de vent...,1378,689.000000,6a77dfe2e50deec474b1373bb7180e39a98bbd0d
904,https://www.francetvinfo.fr/sports/directs/dir...,www.francetvinfo.fr,Euro 2021 : revivez la victoire des Belges sur...,"Pour son quatrième tournoi majeur consécutif, ...",[],"[Pour son quatrième tournoi majeur consécutif,...",2021-06-27T20:14:00,Les tenants du titre portugais abandonnent leu...,NaN,franceinfo: sport,...,NaN,NaN,NaN,NaN,Verif,2,Euro 2021 : revivez la victoire des Belges sur...,828,414.000000,ec2b1e32cbac75d21b7ecfa1ea90d8